In [11]:
import tensorflow as tf
import numpy as np
import imageio

In [12]:
# a toy ResNet model:
inputs = tf.keras.Input(shape = (1024, 1024, 3), name = "img")
x = tf.keras.layers.Conv2D(32, 3, activation="relu")(inputs)
x = tf.keras.layers.Conv2D(64, 3, activation = "relu") (x)
block_1_output = tf.keras.layers.MaxPooling2D(3)(x)

x = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(block_1_output)
x = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_2_output = tf.keras.layers.add([x, block_1_output])

x = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(block_2_output)
x = tf.keras.layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_3_output = tf.keras.layers.add([x, block_2_output])

x = tf.keras.layers.Conv2D(64, 3, activation="relu")(block_3_output)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

outputs = tf.keras.layers.Dense(10)(x)

In [13]:
model = tf.keras.Model(inputs, outputs, name = "toy_resnet")

In [14]:
model.summary()
tf.keras.utils.plot_model(model, "mini_resnet.png", show_shapes = True)

Model: "toy_resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 1024, 1024,  0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 1022, 1022, 3 896         img[0][0]                        
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 1020, 1020, 6 18496       conv2d_14[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 340, 340, 64) 0           conv2d_15[0][0]                  
_________________________________________________________________________________________

In [21]:
X_train = np.zeros((10, 1024, 1024, 3)) # Change to 100 when ready to introduce all data
folder = 'proj_files/train/' 
ext = '.png' 

for i in range(10):
    file = folder + '00000' + str(i) + ext 
    print("reading", file)
    X_train[i, :, :, :] = imageio.imread(file)

# for i in range(10, 100):
#     file = folder + '0000' + str(i) + ext 
#     print("reading", file)
#     X_train[i, :, :, :] = imageio.imread(file)
    
y_train = np.zeros((2, 1024, 1024, 3)) # Change to 100 when ready to introduce all data
folder = 'proj_files/validation/' 
for i in range(2):
    file = folder + '00000' + str(i) + ext 
    print("reading", file)
    y_train[i, :, :, :] = imageio.imread(file)

# for i in range(10, 100):
#     file = folder + '0000' + str(i) + ext 
#     print("reading", file)
#     X_train[i, :, :, :] = imageio.imread(file)


# X_train = tf.convert_to_tensor(X_train)
# y_train = tf.convert_to_tensor(y_train)

y_train = np.zeros((10, 1))

y_train[0:4] = 1

print(np.shape(X_train))
print(np.shape(y_train))



reading proj_files/train/000000.png
reading proj_files/train/000001.png
reading proj_files/train/000002.png
reading proj_files/train/000003.png
reading proj_files/train/000004.png
reading proj_files/train/000005.png
reading proj_files/train/000006.png
reading proj_files/train/000007.png
reading proj_files/train/000008.png
reading proj_files/train/000009.png
reading proj_files/validation/000000.png
reading proj_files/validation/000001.png
(10, 1024, 1024, 3)
(10, 1)


In [22]:

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["acc"],
)

# We restrict the data to the first 1000 samples so as to limit execution time
# on Colab. Try to train on the entire dataset until convergence!
model.fit(X_train, y_train, batch_size=2, epochs=5, validation_split=0.2)

Epoch 1/5


ValueError: in user code:

    /home/ecbm4040/envTF24/lib/python3.6/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /home/ecbm4040/envTF24/lib/python3.6/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ecbm4040/envTF24/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ecbm4040/envTF24/lib/python3.6/site-packages/keras/engine/training.py:835 run_step  **
       